# Use of the Ansible built-in URI method to query a Redfish service

Version 0.160

## Introduction

This Jupyter Notebook uses the Ansible [built-in uri](https://docs.ansible.com/ansible/latest/collections/ansible/builtin/uri_module.html) method and an authentication token for getting and setting parameters of an iLO 5  Redfish service.

The use case of this Notebook explains how you can get and set the properties of an HPE OneView managed compute node without any additional iLO user than the built-in `_HPOneViewAdmin` user.

>**NOTE**: Although HPE Superdome Flex does support OneView SSO, it is interesting to run the Ansible playbook against the SDF simulator The only difference with a real SDF is the authentication mechanism.


In addition, you will learn the basic methodology to discover Redfish resources instead of assuming their final location in the Redfish tree.


## <a id = "EnvironmentPreparation">Environment preparation</a>

The following cell sets environment variables and checks the connectivity to your Synegy iLO 5 simulator.

In [ ]:
########## Environment preparation (Version: 0.134) ############

# Set Student ID number
export stdid={{ STDID }}
Id=$(id --user --name)
NbId=2
InvFile=${NbId}/hosts

# location and ports variables
IloSyBasePort={{ ILOSYBASEPORT }}
let iLO5SimulatorBasePort=$IloSyBasePort
let iLO5SimulatorPort=${iLO5SimulatorBasePort}+${stdid}

iLO5SimulatorIP=ilo5simulators
iLOSimulator=${iLO5SimulatorIP}:${iLO5SimulatorPort}
iLO5SimulatorURI=https://${iLOSimulator}

# Fake Credentials as we are testing against a BMC simulator
OvSsoToken="FakeOvSsoToken"

# Miscellaneous
WorkshopDir=$PWD
HpePythonRedfishVenv="${NbId}/HpePythonRedfishVenv"
export PYTHONPATH="${WorkshopDir}/${NbId}/library/"
w=$(basename $PWD)

alias ResetSimulators="../create-globalbmc.shc.x &>/dev/null ; sleep 1"

# Verify we can reach the remote Bmcs on the right HTTPS ports. 
for bmc in iLO5Simulator ; do
    ip="${bmc}IP" ; port="${bmc}Port" 
    nc -vz  $(eval echo "\$${ip}") $(eval echo "\$${port}") &>/dev/null && 
        echo -e "\n\tGood News: $bmc is reachable" \
        || echo "WARNING: Problem reaching $bmc"
done


# Create the Ansible inventory file
cat > ${InvFile} << __EOF__
[OneViewManagedBmcs]
${iLO5SimulatorIP} ansible_port=${iLO5SimulatorPort}

[OneViewManagedBmcs:vars]
ansible_python_interpreter=${WorkshopDir}/${HpePythonRedfishVenv}/bin/python3
ansible_search_path=${HpePythonRedfishVenv}
# Below is a fake session token as we are testing against an iLO 5 simulator
token="${OvSsoToken}"
__EOF__


# Retrieve iLO firmware versions from ServiceRoot (no credentials needed)
for bmc in iLO5Simulator ; do
    ip="${bmc}IP" ; port="${bmc}Port"
    echo -n -e "\t${bmc} firmware version: "
     curl --silent --insecure -X GET https://$(eval echo \$${ip}):$(eval echo \$${port})/redfish/v1 | \
         jq  '[.Oem.Hpe.Manager[]] | .[] | .ManagerFirmwareVersion'
done

## Virtual Python environment creation

In order to completely isolate this notebook environment from other notebooks or student Python environments, it is safer to create a dedicated Python virtual environment.

> NOTE: This `Venv` creation can take up to **2 minutes**. Just wait until the message `Finished creating Venv` is displayed

In [ ]:
# Create Virtual Python environment (Venv)
[ -d ${HpePythonRedfishVenv} ] && rm -r ${HpePythonRedfishVenv} &>/dev/null 
python3 -m venv ${HpePythonRedfishVenv}                         &>/dev/null
source ${HpePythonRedfishVenv}/bin/activate                     &>/dev/null
export PS1="[PEXP\[\]ECT_PROMPT>"                               # Avoid Venv long prompt messing up outputs

# Install latest Ansible in the Venv
pip install wheel                                               &>/dev/null
pip install jmespath                                            &>/dev/null
pip install ansible                                             &>/dev/null

echo -e "\n\n\tFinished creating Venv\n\n"

### <a id = "iLO5SimulatorRestart">Restart iLO 5 simulator</a>

If you need or desire to restart your iLO 5 simulator in order to restart this workshop from scratch or for other reasons, run the following cell at any time.

In [ ]:
# iLO 5 Simulator restart  (Version 0.1)
ResetSimulators 

# Verify we can reach the remote Bmcs on the right HTTPS ports. 
for bmc in iLO5Simulator ; do
    ip="${bmc}IP" ; port=$(echo ${bmc}Port)
    nc -vz  $(eval echo "\$${ip}") $(eval echo "\$${port}") &> /dev/null && 
        echo "$bmc is reachable" \
        || echo "WARNING: Problem reaching $bmc"
done

## Get and Set Redfish properties using Ansible built-in uri module 

In this section you will change the status of the Unit Identification light (UID, also called `IndicatorLED` in Redfish terminology) of compute nodes in order to facilitate their location in the Datacenter before maintenance. To make this exercise more realistic, you will automatically modify the enclosure UID/LED of these compute nodes if they are part of an enclosure/frame infrastructure.

The `IndicatorLED` resource location is standardized by Redfish as part of the `Chassis` data type and documented in the [HPE Redfish API reference document](https://hewlettpackard.github.io/ilo-rest-api-docs/ilo5/#indicatorled):


![IndicatorLED Chassis property](Pictures/IndicatorLEDProperty.png)


The `Chassis` [data type](https://hewlettpackard.github.io/ilo-rest-api-docs/ilo5/#chassis-v1_6_0-chassis), as per Redfish, is located at `/redfish/v1/Chassis`. From this entry point you will retrieve the `{item}` list of each chassis composing this data type.

In an HPE Synergy compute node, this list is composed of two chassis: a chassis called `enclosurechassis` containing the properties of the frame enclosure, and a chassis called `1` for the compute node.

> **NOTE**: The chassis names are not standadized by Redfish and may change over time.
Moreover, this naming convention is definitively different for Moonshot, Superdome and other vendors of blade computers. Hence, if you want your script to work against other Redfish implementations than ilO based servers, you need to discover each `{item}` in the `Chassis` collection instead of assuming it.

![Chassis data type location](Pictures/ChassisDataType.png)


### Show, modify and verify a Redfish property using Ansible uri

The following cell discovers the `Chassis` collection of the nodes listed in the [hosts](2/hosts) inventory file created in the first cell of this notebook. Then, it prints selected properties of each member of the collection, including the location and the value of the `IndicatorLED`. 

The next tasks toggle the `IndicatorLED` value (`Off` - `Lit`) and apply the modification on each item of the collection. 

The last part of the playbook validates the modification by once again retrieving the `IndicatorLED` property of each item of the chassis collection.

All of the above is performed using the [ansible.builtin.uri](https://docs.ansible.com/ansible/latest/collections/ansible/builtin/uri_module.html) module and a (fake) HPE OneView Single Sign On (SSO) token obtained in the previous notebook.



A convenient way to study the playbook of the next cell is to open it in a different view in this pane. Right click on this Notebook tab name and select `New View for Notebook` to open a new view:
<img src="Pictures/2-NewViewForNotebook.png" align=middle alt="New View for Notebook" height=30px />

Then, click on this [file link](2/SetIndicatorLEDUsingBuiltInUri.yml).

If you need more space, type `Ctrl-B` (or `Command-B` on a Mac) to hide the left pane. You can make it reappear by hitting `Ctrl-B` again.

In [ ]:
# Modify IndicatorLED(s) using the Ansible built-in URI module
ansible-playbook -i ${InvFile} ${NbId}/SetIndicatorLEDUsingBuiltInUri.yml

### Test the same playbook against a rack-mount server

The following cell switches your environment toward an **HPE DL360 Gen10** simulator and then runs again the Ansible Playbook. You will notice that the same playbook works for both a Synergy compute node and a rack-mount server although it is not enclosed in any frame or enclosure chassis.

> **NOTE**: In a real and physical environment, session token authentication against HPE iLO 5 rack mount servers is supported when managed by a OneView appliance. If not managed by OneView, you have to modify the playbook code and supply `url_username` and `url_password` parameters to the `uri` method for basic authentication. Or you could create an additional task with basic authentication to create a session and then extract a session token from the headers of the response of the Redfish service. This mechanism is deeply explained in the [Redfish API 101 Workshop-on-Demand](https://developer.hpe.com) .

In [ ]:
# location and ports variables
IloDlBasePort={{ ILODLBASEPORT }}
let iLO5SimulatorBasePort=$IloDlBasePort
let iLO5SimulatorPort=${iLO5SimulatorBasePort}+${stdid}

iLO5SimulatorIP=ilo5simulators
iLOSimulator=${iLO5SimulatorIP}:${iLO5SimulatorPort}
iLO5SimulatorURI=https://${iLOSimulator}

# Adapt the Ansible inventory file
cat > ${InvFile} << __EOF__
[OneViewManagedBmcs]
${iLO5SimulatorIP} ansible_port=${iLO5SimulatorPort}

[OneViewManagedBmcs:vars]
ansible_python_interpreter=${WorkshopDir}/${HpePythonRedfishVenv}/bin/python3
ansible_search_path=${HpePythonRedfishVenv}
# Below is a fake session token as we are testing against an iLO 5 simulator
token="${OvSsoToken}"
__EOF__


# Modify IndicatorLED(s) using the Ansible built-in URI module against an HPE DL360 Gen10 ilo5 
ansible-playbook -i ${InvFile} ${NbId}/SetIndicatorLEDUsingBuiltInUri.yml

### Test playbook against an HPE Superdome Flex

The following cell switches your environment toward an **HPE Superdome flex RMC** simulator and then runs again the Ansible Playbook.

> **NOTE**: In a real and physical environment, OneView SSO authentication against HPE Superdome Flex RMCs is not supported yet. To have this code work against an HPE Superdome Flex RMC, you have to modify the playbook code and supply `url_username` and `url_password` parameters to the `uri` method for basic authentication. Or you could create an additional task with basic authentication to create a session and then extract a session token from the headers of the response of the RMC Redfish service. This mechanism is deeply explained in the [Redfish API 101 Workshop-on-Demand](https://developer.hpe.com) .

In [ ]:
# location and ports variables
RmcSdBasePort=47000
let RmcSimulatorBasePort=$RmcSdBasePort
let RmcSimulatorPort=${RmcSimulatorBasePort}+${stdid}

RmcSimulatorIP=ilo5simulators
RmcSimulator=${RmcSimulatorIP}:${RmcSimulatorPort}
RmcSimulatorURI=https://${RmcSimulator}

# Adapt the Ansible inventory file
cat > ${InvFile} << __EOF__
[OneViewManagedBmcs]
${RmcSimulatorIP} ansible_port=${RmcSimulatorPort}

[OneViewManagedBmcs:vars]
ansible_python_interpreter=${WorkshopDir}/${HpePythonRedfishVenv}/bin/python3
ansible_search_path=${HpePythonRedfishVenv}
# Below is a fake session token as we are testing against an iLO 5 simulator
token="${OvSsoToken}"
__EOF__

# Modify IndicatorLED(s) using the Ansible built-in URI module against an HPE Superdome Flex RMC 
ansible-playbook -i ${InvFile} ${NbId}/SetIndicatorLEDUsingBuiltInUri.yml

## Summary

In this workshop, you used the Ansible built-in `uri` module to get and set Redfish properties, using an HPE OneView SSO (fake) token and without assuming their location. You validated the same code against three different types of servers, proving its portability (Synergy compute, ProLiant and Superdome Flex). Crawling the Redfish tree using `yaml` is possible, but may become very quickly complex for managing resources deeper than the second level of the Redfish tree.


Select the next [Notebook](3-RedfishAnsibleUsingHpePlaybooks.ipynb) to perform the same exercise with the HPE provided playbooks.